# **Algorithmic Methods of Data Mining - Fall 2022**

## **Homework 5: The Marvel Universe!**

**Packages that are used troughout the notebook:**

In [2]:
# For data analysis and manipulation
import pandas as pd
import numpy as np

# For graph representation
import networkx as nx

# Utils
from tqdm import tqdm
import sys
import scipy

## 1. Data

### Matching hero names
Here we examine and prepare the data so that the hero names match between the two datasets. 

In [3]:
# Load data and strip forward slashes and whitespace
nodes = pd.read_csv('nodes.csv').applymap(lambda x: x.rstrip('/').strip())
edges = pd.read_csv('edges.csv').applymap(lambda x: x.rstrip('/').strip())
network = pd.read_csv('hero-network.csv').applymap(lambda x: x.rstrip('/').strip())

In [4]:
# Unique heroes in each dataframe
node_heroes = set(nodes[nodes['type'] == 'hero']['node'])
edge_heroes = set(edges['hero'])
network_heroes = set(pd.concat([network['hero1'], network['hero2']]))

# Number of unique heroes in each dataframe
len(node_heroes), len(edge_heroes), len(network_heroes)

(6439, 6439, 6421)

We would like to have each hero in edges and network represented in the nodes dataframe. So let's see which ones do not have a match in the nodes dataframe.

In [5]:
print('Heroes in edges but not in nodes:', edge_heroes - node_heroes)
print('Heroes in nodes but not in edges:', node_heroes - edge_heroes)

Heroes in edges but not in nodes: {'SPIDER-MAN/PETER PARKER'}
Heroes in nodes but not in edges: {'SPIDER-MAN/PETER PARKERKER'}


In [6]:
# Fix spiderman's name in nodes
nodes.loc[nodes['node'] == 'SPIDER-MAN/PETER PARKERKER', 'node'] = 'SPIDER-MAN/PETER PARKER'

# Update the heroe set for nodes
node_heroes = set(nodes[nodes['type'] == 'hero']['node'])

# Now let's check the network dataframe
print('Heroes in network but not in nodes:', network_heroes - node_heroes)

Heroes in network but not in nodes: {'SPIDER-MAN/PETER PAR'}


In [7]:
# Fix spiderman's name in network
network = network.applymap(lambda x: 'SPIDER-MAN/PETER PARKER' if x == 'SPIDER-MAN/PETER PAR' else x)
network_heroes = set(pd.concat([network['hero1'], network['hero2']]))

# Now we have every hero in the nodes dataframe
print('Number of heroes in nodes but not in network:', len(node_heroes - network_heroes))

Number of heroes in nodes but not in network: 18


We have processed the data such that all the heroes exist in the nodes df. Now let's drop the rows in the network df which have the same hero pair.

In [8]:
# drop same hero pairs
network = network[network['hero1'] != network['hero2']].reset_index(drop=True)

# Are the comics in the nodes and edges matching?
nodes_comics = set(nodes[nodes['type'] == 'comic']['node'])
edges_comics = set(edges['comic'])
len(nodes_comics), len(edges_comics), len(nodes_comics ^ edges_comics) # The comics are matching

(12651, 12651, 0)

Also, we have discovered that there are some heroes and comics that share the same name. We will add "_c" to the comic name so that it will have its own unique node for the upcoming analysis.

In [9]:
iffy_comics = nodes.loc[nodes['node'].duplicated(keep=False) & (nodes['type'] == 'comic'), 'node']
iffy_comics

2078     BLADE
13362    REBEL
13704    SABRE
Name: node, dtype: object

In [10]:
nodes.loc[nodes['node'].isin(iffy_comics) & (nodes['type'] == 'comic'), 'node'] += '_c'
edges.loc[edges['comic'].isin(iffy_comics), 'comic'] += '_c'

In [11]:
from itertools import combinations
from tqdm import tqdm

# Here we create are own network dataframe
my_network = []

for comic, heroes in tqdm(edges.groupby('comic')['hero']):
    heroes = sorted(heroes)
    for combo in combinations(heroes, 2):

        my_network.append({'hero1': combo[0], 'hero2': combo[1], 'comic': comic})

my_network = pd.DataFrame(my_network)
my_network.shape, network.shape

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 12651/12651 [00:00<00:00, 26130.49it/s]


((579171, 3), (572235, 2))

Lastly, we have discovered that there are some heroes and comics that share the same name. We will add "_c" to the comic name so that it will have its own unique node for the upcoming analysis.

In [12]:
# Delete unnecessary variables
del combo, comic, edge_heroes, edges_comics, heroes, iffy_comics, network_heroes, node_heroes, nodes_comics

## 2. Backend Implementation
We start by creating a list of heroes and their number of appearances so that we can get the top N heroes when needed.

In [13]:
heroes = edges.groupby('hero').size().sort_values(ascending=False)
heroes.head(5)

hero
SPIDER-MAN/PETER PARKER    1577
CAPTAIN AMERICA            1334
IRON MAN/TONY STARK        1150
THING/BENJAMIN J. GR        963
THOR/DR. DONALD BLAK        956
dtype: int64

#### Creating the **First Graph**

We have a bit of an interesting idea to readjust the weights of the edges. You have asked us to have a lower weight/cost for heroes that appear together more frequently. We will add a **small twist** to this idea. These weights will be adjusted based on the number of appearances of the heroes. We will use the following formula to calculate the weight of an edge:
$$
w_{ij} = \frac{|c \in C: h_i \in c| + |c \in C: h_j \in c|}{|c \in C: h_i, h_j \in c|}
$$
where $C$ is the set of all comics, $h_i$ and $h_j$ are two arbitrary heroes in the Marvel Universe. In other words, weight $w_{ij}$ is the inverse of the probability that the two heroes appear together.

In [14]:
edges_weighted = my_network.groupby(['hero1', 'hero2']).size().reset_index().sort_values(by=0, ascending=False)
edges_weighted.columns = ['hero1', 'hero2', 'colab']

edges_weighted['tot'] = edges_weighted['hero1'].map(heroes) + edges_weighted['hero2'].map(heroes)
edges_weighted['weight'] = edges_weighted['tot'] / edges_weighted['colab']
edges_weighted.head(5)

,hero1,hero2,colab,tot,weight
106165,HUMAN TORCH/JOHNNY S,THING/BENJAMIN J. GR,724,1849,2.553867
105742,HUMAN TORCH/JOHNNY S,MR. FANTASTIC/REED R,694,1740,2.507205
141154,MR. FANTASTIC/REED R,THING/BENJAMIN J. GR,690,1817,2.633333
109418,INVISIBLE WOMAN/SUE,MR. FANTASTIC/REED R,682,1616,2.369501
105475,HUMAN TORCH/JOHNNY S,INVISIBLE WOMAN/SUE,675,1648,2.441481


In [15]:
G1 = nx.from_pandas_edgelist(edges_weighted, 'hero1', 'hero2', 'weight')
G1.number_of_nodes(), G1.number_of_edges(), pd.concat([edges_weighted['hero1'], edges_weighted['hero2']]).nunique(), edges_weighted.shape[0]

(6421, 171644, 6421, 171644)

#### Creating the **Second Graph**

In [16]:
G2 = nx.from_pandas_edgelist(edges, 'hero', 'comic')
G2.number_of_nodes(), G2.number_of_edges(), nodes.shape[0], edges.shape[0]

(19090, 96104, 19090, 96104)

In [17]:
G2.add_nodes_from(nodes[nodes['type'] == 'hero']['node'], type='hero')
G2.add_nodes_from(nodes[nodes['type'] == 'comic']['node'], type='comic')
G2.number_of_nodes(), G2.number_of_edges(), nodes.shape[0], edges.shape[0]

(19090, 96104, 19090, 96104)

## 5. Bonus - PageRank on MapReduce

For the implementation of `PageRank` using `PySpark` and MapReduce, we will use the following dataset:

In [23]:
network

,hero1,hero2
0,"LITTLE, ABNER",PRINCESS ZANDA
1,"LITTLE, ABNER",BLACK PANTHER/T'CHAL
2,BLACK PANTHER/T'CHAL,PRINCESS ZANDA
3,"LITTLE, ABNER",PRINCESS ZANDA
4,"LITTLE, ABNER",BLACK PANTHER/T'CHAL
...,...,...
572230,COLOSSUS II/PETER RA,CALLISTO
572231,CALLISTO,ROGUE
572232,CALLISTO,CALIBAN
572233,CALIBAN,ROGUE


First of all, instead of this datafram, we are going to turn our data into `(key, [value])` pairs so that it is easier to implement the PageRank Algorithm.

In [24]:
connections=[]
for i in range(len(network)):
    connections.append((network.iloc[i]['hero1'],[network.iloc[i].hero2]))

In [25]:
links[:10] #This is how our data works now that we have put it in a more convenient way

[('LITTLE, ABNER', ['PRINCESS ZANDA']),
 ('LITTLE, ABNER', ["BLACK PANTHER/T'CHAL"]),
 ("BLACK PANTHER/T'CHAL", ['PRINCESS ZANDA']),
 ('LITTLE, ABNER', ['PRINCESS ZANDA']),
 ('LITTLE, ABNER', ["BLACK PANTHER/T'CHAL"]),
 ("BLACK PANTHER/T'CHAL", ['PRINCESS ZANDA']),
 ('STEELE, SIMON/WOLFGA', ['FORTUNE, DOMINIC']),
 ('STEELE, SIMON/WOLFGA', ['ERWIN, CLYTEMNESTRA']),
 ('STEELE, SIMON/WOLFGA', ['IRON MAN/TONY STARK']),
 ('STEELE, SIMON/WOLFGA', ['IRON MAN IV/JAMES R.'])]

Next, we import our `PySpark` to use MapReduce

In [26]:
import findspark
findspark.init()
findspark.find()
import pyspark
findspark.find()
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
conf = pyspark.SparkConf().setAppName('appName').setMaster('local')
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession(sc)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/01/07 16:38:58 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


We transform our list of pairs created from the data to an equivalent element from `Spark`

In [27]:
conx=spark.sparkContext.parallelize(connections)

In [28]:
conx.collect() #Here we observe that it is indeed the same

23/01/07 16:39:08 WARN TaskSetManager: Stage 0 contains a task of very large size (11279 KiB). The maximum recommended task size is 1000 KiB.


[('LITTLE, ABNER', ['PRINCESS ZANDA']),
 ('LITTLE, ABNER', ["BLACK PANTHER/T'CHAL"]),
 ("BLACK PANTHER/T'CHAL", ['PRINCESS ZANDA']),
 ('LITTLE, ABNER', ['PRINCESS ZANDA']),
 ('LITTLE, ABNER', ["BLACK PANTHER/T'CHAL"]),
 ("BLACK PANTHER/T'CHAL", ['PRINCESS ZANDA']),
 ('STEELE, SIMON/WOLFGA', ['FORTUNE, DOMINIC']),
 ('STEELE, SIMON/WOLFGA', ['ERWIN, CLYTEMNESTRA']),
 ('STEELE, SIMON/WOLFGA', ['IRON MAN/TONY STARK']),
 ('STEELE, SIMON/WOLFGA', ['IRON MAN IV/JAMES R.']),
 ('STEELE, SIMON/WOLFGA', ['RAVEN, SABBATH II/EL']),
 ('RAVEN, SABBATH II/EL', ['FORTUNE, DOMINIC']),
 ('RAVEN, SABBATH II/EL', ['ERWIN, CLYTEMNESTRA']),
 ('RAVEN, SABBATH II/EL', ['IRON MAN/TONY STARK']),
 ('RAVEN, SABBATH II/EL', ['IRON MAN IV/JAMES R.']),
 ('IRON MAN IV/JAMES R.', ['FORTUNE, DOMINIC']),
 ('IRON MAN IV/JAMES R.', ['ERWIN, CLYTEMNESTRA']),
 ('IRON MAN IV/JAMES R.', ['IRON MAN/TONY STARK']),
 ('IRON MAN/TONY STARK', ['FORTUNE, DOMINIC']),
 ('IRON MAN/TONY STARK', ['ERWIN, CLYTEMNESTRA']),
 ('ERWIN, CLYTEMN

In [29]:
#Here we also find the number of nodes
n=conx.count()
n

23/01/07 16:39:22 WARN TaskSetManager: Stage 1 contains a task of very large size (11279 KiB). The maximum recommended task size is 1000 KiB.


572235

In [30]:
# Now we create and initialize the ranks
ranks = conx.map(lambda node: (node[0],1.0/n)) #To every hero we assign the value 1/(num of nodes)
ranks.collect()

23/01/07 16:39:29 WARN TaskSetManager: Stage 2 contains a task of very large size (11279 KiB). The maximum recommended task size is 1000 KiB.


[('LITTLE, ABNER', 1.747533792934721e-06),
 ('LITTLE, ABNER', 1.747533792934721e-06),
 ("BLACK PANTHER/T'CHAL", 1.747533792934721e-06),
 ('LITTLE, ABNER', 1.747533792934721e-06),
 ('LITTLE, ABNER', 1.747533792934721e-06),
 ("BLACK PANTHER/T'CHAL", 1.747533792934721e-06),
 ('STEELE, SIMON/WOLFGA', 1.747533792934721e-06),
 ('STEELE, SIMON/WOLFGA', 1.747533792934721e-06),
 ('STEELE, SIMON/WOLFGA', 1.747533792934721e-06),
 ('STEELE, SIMON/WOLFGA', 1.747533792934721e-06),
 ('STEELE, SIMON/WOLFGA', 1.747533792934721e-06),
 ('RAVEN, SABBATH II/EL', 1.747533792934721e-06),
 ('RAVEN, SABBATH II/EL', 1.747533792934721e-06),
 ('RAVEN, SABBATH II/EL', 1.747533792934721e-06),
 ('RAVEN, SABBATH II/EL', 1.747533792934721e-06),
 ('IRON MAN IV/JAMES R.', 1.747533792934721e-06),
 ('IRON MAN IV/JAMES R.', 1.747533792934721e-06),
 ('IRON MAN IV/JAMES R.', 1.747533792934721e-06),
 ('IRON MAN/TONY STARK', 1.747533792934721e-06),
 ('IRON MAN/TONY STARK', 1.747533792934721e-06),
 ('ERWIN, CLYTEMNESTRA', 1.747

In [ ]:
max_iter=30
for i in tqdm(range(max_iter)):
    # Put together the  graph info with rank info and propogate to all neighbors rank scores (rank/(num of neighbors)
    # and add up ranks from all in-coming edges",
    ranks = l.join(ranks).flatMap(lambda x : [(i, float(x[1][1])/len(x[1][0])) for i in x[1][0]]).reduceByKey(lambda x,y: x+y)
ranks.sortByKey().collect()